<div>
<img src="./img/patreon.PNG" width="600"/>
</div>

# Machine Learning (Introducción)
## Sesión 1

Gabriel Abellán <gabriel.abellan@gmail.com>

En este `notebook` presentaremos algunos términos y técnicas comunes de `machine learning`. Cuando se habla de `Deep Learning`, indicamos un conjunto de herramientas y técnicas de `machine learning` que implican el uso de Redes Neuronales Artificiales.

`Machine Learning` es una rama de la inteligencia artificial que desarrolla algoritmos capaces de aprender patrones y reglas utilizando datos. Aunque conceptualmente es una disciplina bien fundamentada ~1950, su reciente uso exahustivo tiene que ver con tres factores:
- Desarrollo en la capacidad de almacenamiento a bajo costo.
- Desarrollo en la capacidad de computo a bajo costo.
- Desarrollo de dispositivos que producen cantidades enormes de datos (teléfonos moviles, webs, sensores, etc).

El propósito de este taller es introducir conceptos que (tal vez) son nuevos pero aplicarlos a problemas que (tal vez) ya conocen. De esta manera queremos minimizar el impacto del primer encuentro con el tema y acelerar asi la curva de aprendizaje. Una vez familiarizados con la herramienta, se trata de buscar (o crear) nuevos algoritmos e implementarlos dentro del framework.

Los conceptos nuevos tienen que ver con `redes neuronales` y su aplicación; los problemas que trataremos son los viejos y conocidos problemas de `regresión` y `clasificación` (ejemplos de aprendizaje supervisado), así como reducción dimensional haciendo `análisis de componentes principales PCA` (ejemplo de aprendizaje no-supervisado).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import sklearn as skl

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

## Apredizaje Supervisado
### Modelo Lineal - Regresión

Tenemos una data sobre personas que incluye: género, altura y peso. Deseamos hacer un modelo simple que permita predecir el peso en función de la altura.

In [ ]:
data_path = 'https://gitlab.com/gabriel.abellan/machine-learning-la-conga/-/raw/main/datasets/weight-height.csv'

In [ ]:
df = pd.read_csv(data_path)
df.head()

In [ ]:
df.info()   # tambien es posible obtener informacion usando df.describe()
# df.describe()

In [ ]:
df.plot(kind='scatter', x='Height',
        y='Weight', title='Weight Vs. Height in Adults',
        alpha=.4)
plt.plot([55,78], [75,250], color='red', linewidth=2)
plt.show()

definimos una función para construir la ecuación de una recta 1D

In [ ]:
def line(x, w=0, b=0):
    return x*w + b

In [ ]:
x = np.linspace(55,80, 101)
x.shape

Construimos la ecuación de una recta trivial

In [ ]:
yhat = line(x, w=0, b=0)

In [ ]:
df.plot(kind='scatter', x='Height',
        y='Weight', title='Weight Vs. Height in Adults',
        alpha=.4)
plt.plot(x, yhat, color='red', linewidth=2, alpha=.5)
plt.show()

Probamos variando $b$

In [ ]:
df.plot(kind='scatter', x='Height',
        y='Weight', title='Weight Vs. Height in Adults',
        alpha=.4)
plt.plot(x, line(x, b=50), color='orange', linewidth=2, alpha=.5)
plt.plot(x, line(x, b=150), color='red', linewidth=2, alpha=.5)
plt.plot(x, line(x, b=250), color='black', linewidth=2, alpha=.5)
plt.show()

Probamos variando $w$

In [ ]:
df.plot(kind='scatter', x='Height',
        y='Weight', title='Weight Vs. Height in Adults',
        alpha=.4)
plt.plot(x, line(x, w=5), color='orange', linewidth=2, alpha=.5)
plt.plot(x, line(x, w=8), color='red', linewidth=2, alpha=.5)
plt.plot(x, line(x, w=-1), color='black', linewidth=2, alpha=.5)
plt.show()

Es posible describir la data encontrando un buen juego de parametros $(w,b)$

Definimos una función para calcular el error cuadrático medio

In [ ]:
def mean_squared_error(y_true, y_pred):
    s = (y_true - y_pred)**2
    return s.mean()

Queremos hacer un modelo donde se predice el peso de un sujeto usando como predictor la altura

In [ ]:
X = df[['Height']].values
X.shape

In [ ]:
y_true = df['Weight'].values
y_true

Usando el modelo (con los parametros por defecto), calculamos las predicciones para X

In [ ]:
y_pred = line(X)
y_pred

Calculamos el error entre los datos reales y la predicción

In [ ]:
mse_01 = mean_squared_error(y_true, y_pred)
print('mse: {:.3f}'.format(mse_01))

Ahora comenzamos a variar los parametros de la recta y observamos cómo el MSE va cambiando

In [ ]:
y_pred = line(X, w=2)
print('mse: {:.3f}'.format(mean_squared_error(y_true, y_pred.ravel())))

In [ ]:
y_pred = line(X, w=2, b=20)
print('mse: {:.3f}'.format(mean_squared_error(y_true, y_pred.ravel())))

Podemos repetir esto para varios valores de $b$ y graficar

In [ ]:
plt.figure(figsize=(12,5))

ax1 = plt.subplot(121)
df.plot(kind='scatter',
       x='Height', y='Weight', ax=ax1,
       alpha=.4, title='Weight Vs. Height in Adults')

bbs = np.array([-100,-50,0,50,100,150])

mses = []
for b in bbs:
    y_pred = line(X, w=2, b=b)
    mse = mean_squared_error(y_true, y_pred)
    mses.append(mse)
    plt.plot(X, y_pred)
    
ax2 = plt.subplot(122)
plt.plot(bbs, mses, 'o-')
plt.title('Cost as a Function of $b$')
plt.xlabel('$b$')
plt.show()

Este proceso que hemos realizado acá *con la mano* es lo que hace una libreria como `keras` aprovechando los recursos de `Tensorflow`.

En esta presentación hemos decidido usar `keras` de manera que pueda ganarse familiaridad con las herramientas que se utilizan en Deep Learning.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(1, input_shape=(1,)))

In [ ]:
model.summary()

In [ ]:
model.compile(Adam(learning_rate=0.8),loss='mse')
#model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X, y_true, epochs=25, verbose=1)

In [ ]:
y_pred = model.predict(X)
y_pred

In [ ]:
df.plot(kind='scatter',
       x='Height', y='Weight',
       title='Weight Vs. Height in Adults', alpha=.4)
plt.plot(X, y_pred, color='red')
plt.show()

In [ ]:
W, B = model.get_weights()
w = W[0,0]; b = B[0];
print(model.get_weights())
print('w = {:.2f}\nb = {:.2f}'.format(w,b))

Para evaluar modelos de regresión se utiliza la metrica $R^2$.

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r = r2_score(y_true, y_pred)
print('The R^2 score is {:.3f}'.format(r))

## Cross Validation

Podemos usar la librería `sklearn` para implementar el procedimiento de `Cross Validation`

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_true, train_size=.8)

In [ ]:
print(X_train.shape)
print(X.shape)

Reseteamos los parámetros para volver a entrenar

In [ ]:
resetWeights = [np.array([[1.]]), np.array([0.])]
model.set_weights(resetWeights)

In [ ]:
history = model.fit(X_train, y_train, epochs=50, verbose=1, validation_data=(X_test, y_test))

In [ ]:
y_train_pred = model.predict(X_train).ravel()
y_train_pred

In [ ]:
y_test_pred = model.predict(X_test).ravel()
y_test_pred

Evaluamos el performance ahora usando `sklearn`.

In [ ]:
from sklearn.metrics import mean_squared_error as mse

In [ ]:
err = mse(y_train, y_train_pred)
print('Mean Squared Error (Train Set):\t', '{:0.1f}'.format(err))

err = mse(y_test, y_test_pred)
print('Mean Squared Error (Test Set):\t', '{:0.1f}'.format(err))

In [ ]:
r2 = r2_score(y_train, y_train_pred)
print('R2 score (Train Set):\t', '{:0.3f}'.format(r2))

r2 = r2_score(y_test, y_test_pred)
print('R2 score (Test Set):\t', '{:0.3f}'.format(r2))

Es posible usar `history` y observar el comportamiento del entrenamiento

In [ ]:
max_val = pd.DataFrame(history.history).max()
(pd.DataFrame(history.history)/max_val).plot(figsize=(9,5))
#plt.gca().set_ylim(0.99, 1)
plt.grid(True)
plt.show()

Como suele ser la norma, el algoritmo se desempeña mejor sobre el conjunto de entrenamiento que sobre el conjunto de prueba.

Es importante estar atento porque una señal característica de `overfitting` 
es cuando el desempeno continua mejorando sobre el conjunto de entrenamiento
pero se hace peor en el conjunto de prueba. Si esto ocurre hay que revisar.

## Apredizaje Supervisado
### Modelo Lineal - Clasificación Binaria

Queremos predecir si un usuario de cierta página web comprará un producto, usando como dato el tiempo que pasa en la página del producto. La etiqueta es binaria (compró: 1, no compró: 0).

In [ ]:
data_path = 'https://gitlab.com/gabriel.abellan/machine-learning-la-conga/-/raw/main/datasets/user_visit_duration.csv'

df_buy = pd.read_csv(data_path)
df_buy.head()

In [ ]:
df_buy.info()

In [ ]:
df_buy.plot(kind='scatter',x='Time (min)', y='Buy',
           figsize=(8,4))
plt.show()

Definimos las variables predictivas y el tárget

In [ ]:
X = df_buy['Time (min)']
y = df_buy['Buy']

Probamos usar el mismo modelo (arquitectura) que usamos en el ejemplo anterior. Para ello reinicializamos los parámetros.

In [ ]:
resetWeights = [np.array([[1.]]), np.array([0.])]
model.set_weights(resetWeights)

In [ ]:
model.get_weights()

In [ ]:
model.fit(X, y, epochs=200, verbose=0)

In [ ]:
y_pred = model.predict(X)

df_buy.plot(kind='scatter', x='Time (min)', y='Buy',
           title='Linear Fit (Miserably Fail)')
plt.plot(X, y_pred, color='red')
plt.show()

Como puedes ver no tiene mucho sentido utilizar una línea recta para predecir un resultado que sólo puede arrojar valores 0 o 1. Observando esto, la modificación que tenemos que aplicar a nuestro modelo para que funcione es en realidad bastante sencilla.

### Regresión Logístisca

Abordaremos este problema con un método llamado Regresíon Logística. A pesar de que su nombre es "regresión", esta técnica es realmente útil para resolver problemas de clasificación, es decir, problemas en los que el resultado es discreto.

La técnica de regresión lineal que acabamos de aprender predice valores en el eje real para cada punto de datos de entrada. Podemos modificar la forma de la hipótesis para poder predecir la probabilidad de un resultado para cada valor de la entrada. De esta forma nuestro modelo daría un valor entre 0 y 1. En ese punto podríamos utilizar p = 0.5 como criterio de separación y asignar cada punto predicho con una probabilidad inferior a 0.5 a la clase 0, y cada punto predicho con una probabilidad superior a 0,5 a la clase 1.

En otras palabras, si modificamos la hipótesis de regresión para permitir una función no lineal entre el dominio de nuestros datos y el intervalo [0,1], podemos utilizar la misma maquinaria para resolver un problema de clasificación.

Necesitamos una función no lineal que mapee todo el eje real en el intervalo [0,1]. Hay muchas funciones de este tipo. Una función simple, suave y que se comporta bien es la `sigmoide`.

In [ ]:
def sigmoid(z):
    return 1./(1. + np.exp(-z))

z = np.arange(-10, 10, 0.1)

plt.plot(z, sigmoid(z), color='blue')
plt.title('Sigmoid')

Usando la sigmoide podemos formular la hipótesis para el problema de clasificación

$\mbox{Comprar} = \frac{1}{1+e^{-(tw+b})} = \hat{y}$

La sigmoide se utiliza generalmente para la capa de salida en las redes de clasificación. No suele usarse entre capas internas porque hay otras funciones que se comportan mejor.

In [ ]:
x = np.linspace(-10, 10, 100)

plt.figure(figsize=(12,5))

plt.subplot(121)

ws = [.1, .3, 1., 3.]
for w in ws:
    plt.plot(x, sigmoid(line(x, w)))
    
plt.legend(ws)
plt.title('Changing $w$')

plt.subplot(122)

bs = [-5, 0, 5]
for b in bs:
    plt.plot(x, sigmoid(line(x, w=1, b=b)))
    
plt.legend(bs)
plt.title('Changing $b$')
plt.show()

### Función de Costo (loss)

Es necesario ajustar la definición de la función de coste para que tenga sentido para un problema de clasificación binaria. Hay varias opciones para ello, de forma similar al caso de la regresión, incluyendo `square loss`, `hinge loss` y `logistic loss`.

Los modelos de *Deep Learning* aprenden minimizando la función costo. Esto requiere que la función tenga dicho mínimo en primer lugar. En matemáticas, esto quiere decir que la función de coste debe ser convexa y diferenciable.

Una de las funciones de costo mas usadas en *Deep Learning* es `cross entropy`. Esta se define como

$ C_i = -y_i \ln(\hat{y}_i) - (1 - y_i) \ln(1 - \hat{y}_i) $

Dado que $y$ únicamente puede ser 0 o 1, sólo uno de los términos aparece al evaluar.

Si $y_i = 0$

In [ ]:
plt.plot(z, -np.log(1-sigmoid(z)))
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12,4))

axs[0].plot(z, -np.log(1-sigmoid(z)))
axs[0].set_title('$y = 0$')
axs[1].plot(z, -np.log(sigmoid(z)))
axs[1].set_title('$y = 1$')

fig.suptitle('Cross Entropy by Case');

Una vez definida para un solo punto, el promedio de la función costo es

$ c = \displaystyle{\frac{1}{N} \sum_i c_i }$

Esta función puede generalizarse a problemas con múltiples clases. Para ello puede usarse la función `softmax` como generalización de la `sigmoide` y como función costo `categorical cross entropy`.

A continuación definimos nuestro modelo usando `Keras`.

In [ ]:
logistic = Sequential()
logistic.add(Dense(1, input_dim=1))

In [ ]:
from tensorflow.keras.layers import Activation

In [ ]:
logistic.add(Activation('sigmoid'))

In [ ]:
logistic.summary()

In [ ]:
plt.plot(z, logistic.predict(z))
plt.show()

In [ ]:
logistic.compile(optimizer=SGD(learning_rate=.5),
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
logistic.fit(X, y, epochs=25)

In [ ]:
ax = df_buy.plot(kind='scatter', x='Time (min)', y='Buy',
                title='Purchase Vs. Time spent on site')

temp = np.linspace(0,4)
ax.plot(temp, logistic.predict(temp), color='orange')
plt.legend(['model', 'data'])
plt.show()

Notar que la regresión logística produce una probabilidad

In [ ]:
y_pred = logistic.predict(X)
y_pred[:5]

Si queremos una predicción binaria, podemos imponer un umbral a los resultados

In [ ]:
y_pred_bin = y_pred > .5
y_pred_bin[:5].astype(int)

Usando este arreglo podemos calcular el `accuracy` del modelo. Recordamos que `accuracy` es 

$$\mbox{Acc} = \dfrac{TP + TN}{\mbox{All}}$$

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
acc = accuracy_score(y, y_pred_bin)
print('Accuracy Score: {:.3f}'.format(acc))

A continuación realizamos el proceso de validación.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [ ]:
X_test[1:4].values[0]

In [ ]:
params = logistic.get_weights()
params

In [ ]:
params = [np.zeros(w.shape) for w in params]
params

In [ ]:
logistic.set_weights(params)

In [ ]:
acc = accuracy_score(y, logistic.predict(X) > .5)
print('Accuracy Score: {:.3f}'.format(acc))

In [ ]:
plt.plot(z, logistic.predict(z));

In [ ]:
history = logistic.fit(X_train, y_train, epochs=25, verbose=0, validation_data=(X_test, y_test))

In [ ]:
y_train_pred_class = logistic.predict(X_train) > .5
acc = accuracy_score(y_train, y_train_pred_class)
print('Train Accuracy Score: {:0.3f}'.format(acc))

In [ ]:
y_test_pred_class = logistic.predict(X_test) > .5
acc = accuracy_score(y_test, y_test_pred_class)
print('Test Accuracy Score: {:0.3f}'.format(acc))

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10,6))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

Podemos ver algunas predicciones en términos de probabilidades evaluando el modelo

In [ ]:
X_new = X_test[:5]
y_prob = logistic.predict(X_new)

for n in range(X_new.size): 
    print('Para un tiempo de {:.3f} la probabilidad de comprar es {:.2f}.'.format(X_new.values[n], y_prob[n,0]))

### Cross-Validation

Aunque tuviéramos mucho cuidado al dividir nuestros datos de forma aleatoria, esa es sólo una de las muchas formas posibles de realizar una partición de la data. ¿Qué pasaría si realizáramos varias particiones diferentes de entrenamiento/prueba, comprobáramos la puntuación de la prueba en cada una de ellas y, finalmente, promediáramos las puntuaciones? No sólo tendríamos una estimación más precisa del `accuracy` real, sino que también podríamos calcular la desviación estándar de las puntuaciones y, por lo tanto, conocer el error en el `accuracy`. Este proceso se conoce como `cross-validation` y la forma usual de implementarlo es a traves de `K-fold cross-validation`.

En `K-fold cross-validation`, el conjunto de datos se divide en `K` subconjuntos aleatorios de igual tamaño. Entonces, cada uno de los `K` subconjuntos desempeña el papel de conjunto de prueba, mientras que los demás se agregan para formar un conjunto de entrenamiento. De este modo, obtenemos `K` estimaciones de la puntuación del modelo, cada una calculada a partir de un conjunto de pruebas que no se solapa con ninguno de los otros conjuntos de pruebas.

`Scikit-Learn` ofrece `cross-validation` de forma inmediata, pero tendremos que usar un `wrapper` para nuestro modelo y que de esta manera pueda ser entendida por `Scikit-Learn`.

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
pip install scikeras[tensorflow]

In [ ]:
from scikeras.wrappers import KerasClassifier

In [ ]:
def build_logistic():
    logistic = Sequential()
    logistic.add(Dense(1, input_dim=1,
                      activation='sigmoid'))
    logistic.compile(optimizer=SGD(learning_rate=.5),
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
    return logistic

In [ ]:
logistic = KerasClassifier(model=build_logistic,
                          epochs=25, verbose=0)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

Necesitamos poner en forma apropiada el data set

In [ ]:
X_reshape = X.values.reshape(-1,1)
y_reshape = y.values.reshape(-1,1)

In [ ]:
cv = KFold(3, shuffle=True)
scores = cross_val_score(logistic, X_reshape, y_reshape, cv=cv)
scores

In [ ]:
m = scores.mean()
s = scores.std(ddof=1)

print('Cross Validation Accuracy:',
     '{:.4f} ± {:.4f}'.format(m, s))

### Confusion Matrix

¿Es `accuracy` la mejor manera de comprobar el rendimiento de nuestro modelo? Nos dice lo bien que lo estamos haciendo en general, pero no nos da ninguna idea del tipo de errores que está cometiendo el modelo. Veamos como podemos hacerlo mejor.

En el problema que acabamos de introducir, estamos estimando la probabilidad de compra a partir del tiempo de permanencia en una pagina. Se trata de una clasificacion binaria, y podemos acertar o equivocarnos en las cuatro formas representadas aqui:

<div>
    <img src="./img/con_mat.PNG" width="500"/>
</div>

Esta tabla se llama matriz de confusión y da una mejor compresión de las predicciones correctas e incorrectas.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y, y_pred_bin)

Hacemos un poquito de *make-up* para presentarla mejor

In [ ]:
def nice_cm(y_true, y_pred, labels=['False', 'True']):
    cm = confusion_matrix(y_true, y_pred)
    pred_labels = ['Predicted ' + l for l in labels]
    
    df = pd.DataFrame(cm, index=labels,
                     columns=pred_labels)
    return df

In [ ]:
nice_cm(y, y_pred_bin, ['Not Buy', 'Buy'])

Otras métricas para medir el performance son

- `Precision`
- `Recall`
- `F1`

Sus definiciones son: 
$$\mbox{P} = \dfrac{T\!P}{T\!P+F\!P}$$

$$\mbox{R} = \dfrac{T\!P}{T\!P+F\!N}$$

$$\mbox{F1} = \dfrac{2}{\frac{1}{\mbox{P}} + \frac{1}{\mbox{R}}} = 2\dfrac{\mbox{P R}}{\mbox{P + R}}$$

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
precision = precision_score(y, y_pred_bin)
print('Precision:\t{:.3f}'.format(precision))

recall = recall_score(y, y_pred_bin)
print('Recall: \t{:.3f}'.format(recall))

f1 = f1_score(y, y_pred_bin)
print('F1 Score: \t{:.3f}'.format(f1))

## Algunas operaciones comunes que podemos hacer para cambiar la representación de los datos

In [ ]:
df.head()

In [ ]:
pd.get_dummies(df['Gender'], prefix='Gender').head()

In [ ]:
df['Height (feet)'] = df['Height']/12.
df['Weight (100 lbs)'] = df['Weight']/100.

df.describe().round(2)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mms = MinMaxScaler()
df['Weight_mms'] = mms.fit_transform(df[['Weight']])
df['Height_mms'] = mms.fit_transform(df[['Height']])

df.describe().round(2)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
ss = StandardScaler()
df['Weight_ss'] = ss.fit_transform(df[['Weight']])
df['Height_ss'] = ss.fit_transform(df[['Height']])

df.describe().round(2)

In [ ]:
plt.figure(figsize=(15,4))

for (i, feature) in enumerate(['Height', 
                               'Height (feet)', 
                               'Height_mms', 
                               'Height_ss']):
    plt.subplot(1, 4, i+1)
    df[feature].plot(kind='hist', title=feature)
    plt.xlabel(feature)
    
plt.tight_layout()

### Ejercicio 1:

- cargar la data `housing-data.csv` (*aquí la data* <https://gitlab.com/gabriel.abellan/machine-learning-la-conga/-/raw/main/datasets/housing-data.csv>)
- graficar los histogramas para cada feature
- crear dos variables `X`, `y`: `X` debe ser una matriz con tres columnas (sqft, bdrms, age); `y` debe ser un vector con una columna (price)
- configurar un modelo en Keras para hacer un ajuste lineal (tener cuidado del número apropiado de inputs y outputs)
- dividir la data en `train` y `test`
- entrenar el modelo usando el conjunto `train` y contrastar el desempeño obtenido tanto con `train` como con `test`
- ¿cómo se comporta el modelo?
- trata de mejorar el modelo realizando alguna de las siguientes acciones:
    - normalizar los inputs con algunas de los metodos mencionados
    - usa un valor de `learning rate` diferente
    - usar un `optimizer` distinto
- cuando estés satisfecho con el resultado, calcula el valor $R^2$ sobre el `test`

**Opcional 1**

Una vez que has encontrado un modelo con el cual estás satisfecho, es posible hacer un test simple y convincente para determinar cuán bueno es el performance del modelo. La idea es la siguiente:
- Tomar el vector `y_train` y realizar una permutación.
- Entrenar el modelo usando este nuevo `y_train_nuevo` permutado.
- Calcular el `mean_squared_error` (o $R^2$ para efectos del ejercicio es indiferente) y guardarlo en una lista.
- Iterar sobre los tres pasos anteriores (100, 1000 veces? Lo que te permita la máquina en un tiempo razonable; razonable quiere decir unos pocos minutos).
- Con la lista creada haz un histograma.
- Determina la probabilidad de obtener el desempeño original en términos de esta distribución.

Lo que hacemos aca es esencialmente un test de hipótesis y lo que estamos respondiendo es cuál es la probabilidad de que el desempeño obtenido inicialmente sea producto del azar.

**Opcional 2**

Transformar el problema de regresión a un problema de clasificación. La idea es:
- Dividir los valores en `y` (price) en categorías. Para ello hay que realizar cortes y asignarles una etiqueta.
- Replantear el problema para atacarlo como un problema de clasificacion: decidir la función costo, decidir la métrica, etc.

### Ejercicio 2

El objetivo es predecir la variable `left` usando el resto de la data. Como el target es binario, este es un problema de clasificación.

- cargar la data `HR_comma_sep.csv` (*aqui la data* <https://gitlab.com/gabriel.abellan/machine-learning-la-conga/-/raw/main/datasets/HR_comma_sep.csv>). Inspeccionar.
- verificar si algún feature necesita reescalamiento.
- transformar las variables categóricas usando dummies.
- split la data.
- jugar con los ajustes del learning rate y el optimizer.
- verificar la matriz de confusion, precision y recall.
- verificar los resultados usando 5-Fold cross-validation.
- ¿es bueno este modelo?

***Opcional 1***

Para tratar de refinar el modelo podemos tratar de explorar las variables que funcionan mejor como predictores. Podemos usar dos técnicas simples tratando de determinar el grado de correlación.
1. Si la correlación es categórica-continua hacemos un modelo logístico exploratorio para el `target` y cada variable numérica que queremos explorar. Si hay correlación, lograremos producir un buen modelo predictivo con ese par.
2. Si la correlación es categorica-categorica lo más simple es construir una tabla de contingencia (`cross tabulation`) y detectar de allí si hay alguna correlación.

La idea final es quedarse con las variables que produzcan los mejores resultados.
- Una vez que hayas reducido el espacio de variables predictivas, contruye un nuevo modelo y compara el performance con el obtenido usando todas las variables.
- ¿Qué puedes concluir?

***Opcional 2***

Otra forma de refinar el modelo es introducir una capa oculta al modelo de clasificación.

- Modifica el modelo original (o el de la ***opcion 1***) y agrega una capa (oculta) entre la entrada y la salida. Esta capa sera de la clase `Dense` al igual que las otras que hemos trabajado.
- Experimenta con el numero de neuronas de la capa oculta. Investiga sobre las funciones de activación.
- Entrena el modelo y compara con los resultados obtenidos en el modelo original.
- ¿Qué puedes concluir?